<a href="https://colab.research.google.com/github/JamesMartinOU/PublicRedditSentimentAnalysis/blob/main/RedditCommentsToDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section One - Python packages, variables, and credentials:
1.   Install Python packages
2.   Import Python packages
3.   Define global variables, Reddit API credentials, and MySQL credentials

In [ ]:
# Install Python packages
!pip install praw
!pip install pandas
!pip install mysql-connector-python
!pip install gspread oauth2client

In [ ]:
# Import Python packages
import pandas as pd
import mysql.connector
import praw
import warnings
from google.colab import files
import time
import prawcore
import openpyxl
from google.colab import auth
import gspread
from google.auth import default

In [ ]:
# Define global variables, API and database credentials

# Master list of Post IDs that have 0 fetchable comments and query post ids
master_no_comments_list = []
query_post_ids_list = []


# Section Two - Import Google sheets "no_comment_posts"

1.   Import Google Sheets file - "no_comment_posts" containing lists of post ids without comments.



In [ ]:
# Import google sheet containing list of posts without comments

auth.authenticate_user()
# Get default credentials from google-auth
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet (by name) and select the first worksheet
sheet = gc.open("no_comment_posts").sheet1

# Fetch all data (list of lists)
all_values = sheet.get_all_values()

# Extract the first column, skipping the header
master_no_comments_list = [row[0] for row in all_values[1:]]

# Confirm the list
print("no_comment_posts:", master_no_comments_list)



no_comment_posts: ['11vwzdi', '11xsx5z', '13u1ezc', '15idj4l', '17cavf0', '18luh82', '1bc3m6k', '1d4vh7a', '1din5fa', '1djew24', '1dp2r49', '1dr71l4', '1dyhdat', '1ek3l5w', '1eqkztk', '1ffc5qd', '1fj631t', '1fr9rdg', '10lo05u', '10yrxai', '1g9lr4u', '1il6mr8', '7epuff', '7p79jq', '7zwr77', '7zwrkr', '8f3rkt', 'aer7t3', 'aknvdq', 'bhabvq', 'cufvzv', 'ewhem8', 'f28a1w', 'gv7oqj', 'gvuoh7', 'ko9bng', 'n18ch8', 'ndgy0d', 'p4u8lm', 'qp46a3', 'r6ff3k', 's3o7zs', 'sa9f6o', 'sh1q49', 'tnt9u6', 'u7k6tp', 'u95ikw', 'w9dkp3', 'wlqinv', 'wscoge', 'xavuxr', 'xku9yi', 'y3tcye', 'y9smuq', 'y9wwsr', 'yc8195', 'ycenlw', 'yk08g6']


# Section Three - Define functions:

1.   Define get_comments function - retrieves Reddit comments for provided post_id.
2.   Define delete_comments_from_last_post function - deletes all comments pertaining to the last post_id added to reddit_comments table.
3.   Define write_missing_comments function - writes top 15 Reddit comments for post_id to database table reddit_comments.
4.   Define get_post_ids_with_comments function - retrieves list of post_ids that are queryable, by removing post_ids with zero comments and post_ids already in reddit_comments table.



In [ ]:
# Function to retrieve comments attached to post id
def get_comments(post_id):
    post = reddit.submission(id=post_id)
    comments = []
    time.sleep(1)
    for comment in post.comments.list(): # Iterate through the entire comment forest (including MoreComments)
        if isinstance(comment, praw.models.MoreComments):  # Check if it's a MoreComments object
            continue  # Skip MoreComments objects
        if comment.depth == 0:  # Only include depth 0 comments
            comments.append({
                'post_id': post_id,
                'comment_id': comment.id,
                'author': comment.author.name if comment.author else "No Author",
                'score': comment.score,
                'created_utc': comment.created_utc,
                'body': comment.body,
                'parent_id': comment.parent_id,  # Parent (either post or another comment)
                'permalink': f"https://www.reddit.com{comment.permalink}",  # Direct link to comment
                'depth': comment.depth,  # Depth in the comment tree
                'is_submitter': comment.is_submitter,  # If author is OP
                'edited': comment.edited if isinstance(comment.edited, bool) else time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(comment.edited)) if comment.edited else None,
                'controversiality': comment.controversiality  # Indicates if the comment is controversial
            })

            if len(comments) >= 10:  # Stop collecting after 15 comments
                break

    return comments

In [ ]:
# Function to delete all comments associated with last post entered into comments table
def delete_comments_from_last_post():
    try:
        # Establish connection to MySQL
        conn = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        cursor.execute("SELECT COUNT(*) FROM reddit_comments;")
        record_count = cursor.fetchone()[0]  # Fetch the count from the query result

        cursor.execute("SELECT DISTINCT post_id FROM reddit_comments;")
        comment_post_ids_list = cursor.fetchall()
        # Extract post IDs into a list
        comment_post_ids_list = [post_id[0] for post_id in comment_post_ids_list]

        # Now select the post_id from the last record
        cursor.execute(f"""
            SELECT post_id
            FROM reddit_comments
            LIMIT 1 OFFSET {record_count - 1};
        """)

        last_post_id = cursor.fetchall()
        last_post_id = [post_id[0] for post_id in last_post_id]

        if last_post_id:
            # Delete all comments associated with the last post_id
            cursor.execute("""
                DELETE FROM reddit_comments
                WHERE post_id = %s
            """, (last_post_id[0],))

            conn.commit()
            print(f" {record_count} records in comment table.")
            print(f"✅ All comments for post ID {last_post_id[0]} have been deleted.")
            print(comment_post_ids_list)
        else:
            print("❌ No posts found in the comments table.")

    except mysql.connector.Error as err:
        print(f"❌ Error: {err}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("🔌 MySQL connection closed.")


In [ ]:
def write_missing_comments(post_ids_list):
    """
    Fetches comments for each post in post_ids_list and writes them to a MySQL table.
    Additionally, for posts that have no comments, the post ID is appended to an existing Google Sheets file.

    Parameters:
    - post_ids_list: List of Reddit post IDs.
    """
    try:
        # Connect to MySQL and define insert query
        conn = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        insert_query = """
            INSERT INTO reddit_comments (post_id, comment_id, author, score, created_utc, body, parent_id, permalink, depth, is_submitter, edited, controversiality)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Loop through each post_id in post_ids_list
        for post_id in post_ids_list:
            # Execute get comments function
            print(f"📥 Fetching comments for post ID: {post_id}")
            comments = get_comments(post_id)
            # Post without comments operations and append to google sheets
            if not comments:
                print(f"⚠️ No comments found for post ID {post_id}")
                master_no_comments_list.append(post_id)
                sheet.append_row([post_id])
                print(f"{post_id} appended to Google Sheets list")
                continue

            # Define comments data
            comments_data = [
                (
                    comment['post_id'],
                    comment['comment_id'],
                    comment['author'],
                    comment['score'],
                    time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(comment['created_utc'])),
                    comment['body'],
                    comment['parent_id'],
                    comment['permalink'],
                    comment['depth'],
                    comment['is_submitter'],
                    comment['edited'],
                    comment['controversiality']
                )
                for comment in comments
            ]

            # Insert comments into MySQL
            cursor.executemany(insert_query, comments_data)
            conn.commit()
            print(f"✅ Inserted {len(comments_data)} comments for post ID {post_id}")
            time.sleep(1)  # Sleep to avoid rate limits

        print("🎉 All missing comments fetched and inserted successfully.")

    except mysql.connector.Error as err:
        print(f"❌ MySQL Error: {err}")

    except praw.exceptions.PRAWException as e:
        print(f"❌ Reddit API Error: {e}")

    finally:
        # Close MySQL connection
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print("🔌 MySQL connection closed.")


In [ ]:
# Retrieve post ids for posts that do not have comments in comments table
def get_post_ids_with_comments():
# First delete incomplete comment records for last post
  delete_comments_from_last_post()

  try:
      # Establish connection to MySQL
      conn = mysql.connector.connect(
          host=DB_HOST,
          user=DB_USER,
          password=DB_PASSWORD,
          database=DB_NAME
      )

      cursor = conn.cursor()

      # Query to select all post ids from the reddit_comments table
      cursor.execute("SELECT DISTINCT post_id FROM reddit_comments;")
      # Fetch all results
      post_ids = cursor.fetchall()
      post_ids_comments_table_list = [post_id[0] for post_id in post_ids]
      print(f"{len(post_ids_comments_table_list)} posts completely housed in comments table")

      # Query to select all post ids not in reddit_comments table
      cursor.execute("SELECT id FROM reddit_posts WHERE id NOT IN (SELECT DISTINCT post_id FROM reddit_comments) and num_comments >= 1;")
      post_ids = cursor.fetchall()
      post_ids_list = [post_id[0] for post_id in post_ids]

      # Filter out post ids that do not have comments
      try:
          if len(master_no_comments_list) > 0:
            query_post_ids_list = [post_id for post_id in post_ids_list if post_id not in master_no_comments_list]
          else:
            query_post_ids_list = post_ids_list
      except:
          print("No posts with no comments found")

      print(f"{len(post_ids_list)} posts not completely housed in comments table")
      print(f"{len(master_no_comments_list)} posts without comments")
      print(f"{len(query_post_ids_list)} posts to query")

  except mysql.connector.Error as err:
      print(f"❌ Error: {err}")

  finally:
      if 'conn' in locals() and conn.is_connected():
          cursor.close()
          conn.close()
          print("🔌 Connection closed.")

  # Return a list of posts to query
  return query_post_ids_list



# Section Four - Execute functions to begin writing comments to reddit_comments MySQL table:

1.   Run get_post_ids_with_comments function and assign queryable list of post_ids to query_post_ids_list.
2.   Run write_missing_comments function - writes top 15 comments for each
post_id in query_post_ids_list.



In [ ]:
# Write post comments to reddit_comments table
query_post_ids_list = get_post_ids_with_comments()
print(query_post_ids_list)
write_missing_comments(query_post_ids_list)

 909536 records in comment table.
✅ All comments for post ID 1jgzg5w have been deleted.
['6rabh', '6rp37', '6t6d5', '6tg9b', '70gfl', '70q48', '71tz5', '71woz', '72svy', '72tjc', '73htx', '74m7g', '75r0g', '75uen', '75xb2', '787e3', '85363', '878fu', '9aaia', '9js46', '9sbzh', 'ab6e5', 'awp53', 'azi31', 'b8tf1', 'b9ofu', 'baa6u', 'bhtf5', 'brf2o', 'd7u6u', 'dicn0', 'djq98', 'ds8q3', 'egr7o', 'exm0o', 'fca3k', 'fq96a', 'ftat1', 'fyuzk', 'gv87x', 'h1cjr', 'hi5gf', 'hpvig', 'i29pq', 'i62ig', 'jbidk', 'jjk7g', 'jn7hg', 'jyl3u', 'k0zuj', 'l9xvz', 'lijv5', 'ls2ek', 'm7bvd', 'm80x2', 'mezqq', 'nfd81', 'ng9hi', 'nlacj', 'nrayd', 'nray9', 'nvpim', 'o6w5g', 'os9lg', 'qh3h3', 'qkqvx', 'qru56', 'qy086', 'qz8dl', 'rghrs', 'rikg1', 'rmwi9', 'saqvi', 'scbf8', 'scuq6', 'sobhg', 'szln9', 't0o94', 'ukj1t', 'uxpil', 'v1kpn', 'v2clh', 'vdhwj', 'vhuz6', 'vpndn', 'vt6eg', 'vvtlw', 'vvwar', 'w4fwu', 'w6n4k', 'wmu2w', 'x319b', 'x6x29', 'xmuqc', 'y542u', 'yi8rm', 'yv4a1', 'z14wb', 'zm11f', 'zpew0', '10ft8z', '

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



✅ Inserted 5 comments for post ID 1jgzg5w
🎉 All missing comments fetched and inserted successfully.
🔌 MySQL connection closed.
